# Simulating Deletions

In [1]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import \
    single_gene_deletion, single_reaction_deletion, \
    double_gene_deletion, double_reaction_deletion

cobra_model = cobra.test.create_test_model("textbook")
ecoli_model = cobra.test.create_test_model("ecoli")

## Single Deletions

Perform all single gene deletions on a model

In [2]:
growth_rates, statuses = single_gene_deletion(cobra_model)

These can also be done for only a subset of genes

In [3]:
gr, st = single_gene_deletion(cobra_model,
                              cobra_model.genes[:20])
pandas.DataFrame.from_dict({"growth_rates": gr,
                            "status": st})

,growth_rates,status
b0116,0.782351,optimal
b0118,0.873922,optimal
b0351,0.873922,optimal
b0356,0.873922,optimal
b0474,0.873922,optimal
b0726,0.858307,optimal
b0727,0.858307,optimal
b1241,0.873922,optimal
b1276,0.873922,optimal
b1478,0.873922,optimal


This can also be done for reactions

In [4]:
gr, st = single_reaction_deletion(cobra_model,
                                  cobra_model.reactions[:20])
pandas.DataFrame.from_dict({"growth_rates": gr,
                            "status": st}).round(4)

,growth_rates,status
ACALD,0.8739,optimal
ACALDt,0.8739,optimal
ACKr,0.8739,optimal
ACONTa,0.0000,optimal
ACONTb,0.0000,optimal
ACt2r,0.8739,optimal
ADK1,0.8739,optimal
AKGDH,0.8583,optimal
AKGt2r,0.8739,optimal
ALCD2x,0.8739,optimal


## Double Deletions

Double deletions run in a similar way. Passing in return_frame=True will cause them to format the results as a pandas Dataframe

In [5]:
double_gene_deletion(cobra_model, cobra_model.genes[-5:],
                     return_frame=True).round(4)

,b2464,b0008,b2935,b2465,b3919
b2464,0.8739,0.8648,0.8739,0.8739,0.704
b0008,0.8648,0.8739,0.8739,0.8739,0.704
b2935,0.8739,0.8739,0.8739,0.0000,0.704
b2465,0.8739,0.8739,0.0000,0.8739,0.704
b3919,0.7040,0.7040,0.7040,0.7040,0.704


By default, the double deletion function will automatically use multiprocessing, splitting the task over up to 4 cores if they are available. The number of cores can be manually sepcified as well. Setting use of a single core will disable use of the multiprocessing library, which often aids debuggging.

In [6]:
start = time()  # start timer()
double_gene_deletion(ecoli_model, ecoli_model.genes[:300],
                     number_of_processes=2)
t1 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 2 cores" % t1)

start = time()  # start timer()
double_gene_deletion(ecoli_model, ecoli_model.genes[:300],
                     number_of_processes=1)
t2 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 1 core" % t2)

print("Speedup of %.2fx" % (t2 / t1))

Double gene deletions for 200 genes completed in 27.03 sec with 2 cores
Double gene deletions for 200 genes completed in 40.73 sec with 1 core
Speedup of 1.51x


Double deletions can also be run for reactions

In [7]:
double_reaction_deletion(cobra_model,
                         cobra_model.reactions[2:7],
                         return_frame=True).round(4)

,ACKr,ACONTa,ACONTb,ACt2r,ADK1
ACKr,0.8739,0.0,0.0,0.8739,0.8739
ACONTa,0.0000,0.0,0.0,0.0000,0.0000
ACONTb,0.0000,0.0,0.0,0.0000,0.0000
ACt2r,0.8739,0.0,0.0,0.8739,0.8739
ADK1,0.8739,0.0,0.0,0.8739,0.8739
